In [1]:
import json
import os
from collections import defaultdict

In [2]:
path = "/scratch/zar8jw/Audio/accumulate_segments"
res = defaultdict(dict)
for root, dirs, files in os.walk(path):
    name = os.path.basename(root)
    for file in files:
        if file == "transcript.json":
            text = ""
            with open(os.path.join(root, file), 'r') as f:
                cur_transcripts = json.load(f)
            for i in range(len(cur_transcripts)):
                text += cur_transcripts[i]["transcript"]
            res[name]["pred"] = text

In [3]:
res

defaultdict(dict,
            {'GX010332_encoded_trimmed': {'pred': 'You guys are going to know how to do CPR after watching it 500 times.Check pulse.  Start CPR.Check pulse. Start CPR.Patient is puking everywhere.Are you okay?Checking for a pulse.Check breath.  Pppppp...No breaths.  Initiating CPR.Starting CPR. One.  Pppppp...No pulse. No pulse.  Initiating CPR.Applying AED pads. Analyzing rhythm. Shock advised.  Delivering shock.  Checking for pulse.  Starting CPR.  One... two... three... four...Analyzing heart rhythm.Analyzing heart rhythmShock advisedDo not touch the patient.Everyone clear. Initiating CPR.Press the flashing shock button.Shock delivered.Begin CPR.  Pppppp... No pulse. No breaths. Initiating CPR.Give two breaths.'},
             'GX010322_encoded_trimmed': {'pred': 'checking pulseno pulse, no breathingStarting CPRGiving two breaths.Starting CPR. Giving two breaths.Stop CPR.  [Sound of AED]Do not touch the patient. Analyzing heart rhythm. [Sound of AED]Do not touch th

In [4]:
file_names = [each.replace("_trimmed", "_human.json") for each in list(res.keys())]
path = "/scratch/zar8jw/Audio/manual_check_transcripts"
for root, dirs, files in os.walk(path):
    for file in files:
        if file in file_names:
            with open(os.path.join(root, file), 'r') as f:
                gt_file = json.load(f)
            
            gt_text = ""
            for i in range(len(gt_file)):
                gt_text += gt_file[i]["Utterance"] + " "
            
            res[file.replace("_human.json", "_trimmed")]["gt"] = gt_text

In [5]:
res

defaultdict(dict,
            {'GX010332_encoded_trimmed': {'pred': 'You guys are going to know how to do CPR after watching it 500 times.Check pulse.  Start CPR.Check pulse. Start CPR.Patient is puking everywhere.Are you okay?Checking for a pulse.Check breath.  Pppppp...No breaths.  Initiating CPR.Starting CPR. One.  Pppppp...No pulse. No pulse.  Initiating CPR.Applying AED pads. Analyzing rhythm. Shock advised.  Delivering shock.  Checking for pulse.  Starting CPR.  One... two... three... four...Analyzing heart rhythm.Analyzing heart rhythmShock advisedDo not touch the patient.Everyone clear. Initiating CPR.Press the flashing shock button.Shock delivered.Begin CPR.  Pppppp... No pulse. No breaths. Initiating CPR.Give two breaths.',
              'gt': "I'm sorry. Yeah, I disconnected from the wifi. Maybe because I got a new person. I'm sorry.  It just happens sometimes. I have some. Actually, yeah. I should have some, I think. I'm so sorry. This thing usually works fine. No. I even g

In [6]:
import jiwer
from collections import defaultdict
import inflect
import string
# Create an inflect engine
p = inflect.engine()

# Define a custom function to replace digits with words
def digits_to_words(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = text.split()
    converted_words = [p.number_to_words(word) if word.isdigit() else word for word in words]
    return " ".join(converted_words)

def get_wer(gt, pred):

    gt_transformed = digits_to_words(gt)
    pred_transformed = digits_to_words(pred)

    transforms = jiwer.Compose(
        [
            jiwer.ExpandCommonEnglishContractions(),
            jiwer.RemoveEmptyStrings(),
            jiwer.ToLowerCase(),
            jiwer.RemoveMultipleSpaces(),
            jiwer.Strip(),
            jiwer.RemovePunctuation(),
            jiwer.ReduceToListOfListOfWords(),
        ]
    )
    wer = jiwer.wer(
                gt_transformed,
                pred_transformed,
                truth_transform=transforms,
                hypothesis_transform=transforms,
            )
    return wer

In [7]:
import numpy as np
all_wer = []
for k, metadata in res.items():
    wer = get_wer(metadata["gt"], metadata["pred"])
    all_wer.append(wer)
    print(k, wer)
print(f"avg: {np.average(all_wer)}")

GX010332_encoded_trimmed 0.8844444444444445
GX010322_encoded_trimmed 0.7536231884057971
GX010325_encoded_trimmed 1.7936507936507937
GX010319_encoded_trimmed 0.5473684210526316
GX010336_encoded_trimmed 0.9893617021276596
GX010324_encoded_trimmed 0.8
GX010323_encoded_trimmed 1.0714285714285714
GX010364_encoded_trimmed 2.378640776699029
GX010321_encoded_trimmed 0.9459459459459459
GX010318_encoded_trimmed 0.72
avg: 1.0884463843754872
